# <font color=red>LangChain:  Prompts</font>
- https://docs.langchain.com/docs

## What Does LangChain Provide?
+ Models
  + embedding
  + LLM (e.g. OpenAI)
<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
+ Prompts
  + prompt templates
  + few-shot
  + example-selectors
  + output parsers
</font></span>
+ Chains (a multi-step workflow composed of <em>links</em>)</br>
  + Links (one of: prompt, model, another chain)
+ Vector Database Access
  + Document Loaders
  + Text Splitting 
+ Memories (to facilitate chatbots or other 'iterative' sorts of apps)
+ Agents (loop over Thought, Act, Observe)
  + Tools
    + math
    + web search
    + custom (user-defined)

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Prompts
</font></span>
<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
### Prompt Templates
</font></span>

In [ ]:
## make sure your OpenAI key is in the environment
from langchain import PromptTemplate

# first way to create a prompt template (constructor)
constructor_prompt = PromptTemplate(
    input_variables=["joke_type", "topic"],
    template="Tell me a {joke_type} joke about {topic}.",
)

# second way to create a prompt template (helper method); I like this one :-)
template = "Tell me a {joke_type} joke about {topic}."
helper_prompt = PromptTemplate.from_template(template)

print(f"constructor: {constructor_prompt.format(joke_type='funny', topic='chickens')}")
print( f"helper method: {helper_prompt.format(joke_type='funny', topic='chickens')}")


<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
### Few-Shot Prompts
</font></span>

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate

examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall",  "antonym": "short"},
    {"word": "sharp", "antonym": "dull"},
]

example_template = """word: {word}
antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input\n",
    suffix="word: {input}\nantonym: ",
    input_variables=["input"],
    example_separator="\n",
)
print(few_shot_prompt.format(input="small"))  # NOTE: we are only printing the prompt


<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
### Example Selectors (setting a max len for samples)
</font></span>

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall",  "antonym": "short"},
    {"word": "sharp", "antonym": "dull"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template="word: {word}\nantonym: {antonym}",
)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,  # max len of text of formatted examples
)

dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="give the antonym of all inputs",
    suffix="word: {input}\nantonym:",
    input_variables=["input"],
)

print(dynamic_prompt.format(input="big"))  # NOTE: we just print the prompt

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
### Output Parsers
</font></span>
We will cover output parsers here for lack of a better place. </br>
I have not used output parsers much, but thought they need to be included for completeness.</br>
Below, is a demo from LangChain of a parser that extracts the useful info and produces it in JSON format.

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

model = OpenAI(model_name="text-davinci-003", temperature=0.0)

# joke data structure
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

# setup a parser + inject instructions into the prompt template
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
## this print produces a very long line of output (prompt describes producing JSON output)
print(f"BEGIN PROMPT\n{prompt}\nEND PROMPT")

joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)
output = model(_input.to_string())
out = parser.parse(output)
print(out)